In [1]:
import pandas as pd

In [2]:
df_stages = pd.read_csv("TDF_Stages_History.csv")
df_etapes = pd.read_excel("dates_etapes.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'TDF_Stages_History.csv'

In [ ]:
# Création d'une colonne de clé de fusion dans df_etapes
# Concaténation de l'année, du départ et de l'arrivée
df_etapes['Stage_Key'] = df_etapes['annee'].astype(str) + ' : ' + df_etapes['depart'] + ' > ' + df_etapes['arrivee']

# Extraction des villes de départ et d'arrivée de la colonne 'Stage' dans df_stages
# La clé inclura également l'année
df_stages['Stage_Key'] = df_stages['Year'].astype(str) + ' : ' + df_stages['Stage'].str.extract(r'Stage \d+ : (.+?) > (.+?)$')[0] + ' > ' + df_stages['Stage'].str.extract(r'Stage \d+ : (.+?) > (.+?)$')[1]

# Fusion des DataFrames sur la colonne 'Stage_Key'
merged_df = pd.merge(df_etapes, df_stages, on='Stage_Key', how='inner')
merged_df = merged_df.dropna(axis=1, how='all')

# Affichage des premières lignes du DataFrame fusionné pour vérification
merged_df

In [ ]:
#je supprime les colonnes redondantes suite au merged des 2 bases de données
merged_df = merged_df.drop(columns=['Year', 'Stage', 'Stage_Key'])
merged_df 

Categoriser la colonne 'weather'

Recherche des attributs unique de la colonne 'Weather'

In [ ]:
L=[]
attributs_uniques = merged_df['Weather'].unique()

print("Attributs uniques dans la colonne 'Weather':")
for attribut in attributs_uniques:
    print(attribut)
    L.append(attribut)

In [ ]:
# Dictionnaire des catégories et leurs mots-clés associés
categories = {
    'Categorie_Beau_Temps': ["beau temps", "temps ensoleillé", "grand soleil", "soleil", "ciel bleu", "ciel dégagé", "soleil ardent", "temps clair", "ensoleillé", "grand soleil", "temps radieux", "beau soleil", "soleil de plomb", "beau temps chaud", "ciel nuageux à dégagé", "temps sec"],
    'Categorie_Pluvieux': [ "temps orageux", "pluie", "pluies intermittentes", "temps pluvieux", "orage", "pluie battante", "pluies persistantes", "temps maussade", "temps gris et pluvieux", "temps couvert et pluvieux", "temps frais et pluvieux", "pluie ininterrompue", "pluie violente", "temps maussade et pluvieux", "temps orageux et pluvieux"],
    'Categorie_venteux' : ["ciel nuageux, temps assez couvert", "temps couvert", "temps gris, ciel gris","temps nuageux à couvert", "temps couvert et menaçant", "temps couvert avec éclaircies", "temps couvert et frais", "temps couvert et chaud", "temps gris et couvert", "temps couvert au départ", "temps nuageux mais sec", "temps couvert avec quelques averses"],
    'Categorie_Caniculaire': ["chaud et ensoleillé", "chaleur", "temps chaud et lourd", "caniculaire", "chaleur accablante", "temps très chaud", "forte chaleur", "temps chaud", "chaleur torride", "chaleur lourde", "très forte chaleur", "temps chaud et beau"],
    'Categorie_Changeant' : ["temps variable", "alternant ondées et soleil", "temps changeant", "temps variable à beau", "temps variable et venteux", "temps variable à couvert", "temps variable et doux", "temps variable", "sec", "temps frais et variable"],
    'Categorie_Autre' : [] 
}

def categoriser_meteo(texte):
    texte = texte.lower()
    for categorie, mots_cles in categories.items():
        if any(mot in texte for mot in mots_cles):
            return categorie
    return 'Autre_Categorie'

# Appliquer la fonction à la colonne 'meteo'
merged_df['meteo_categorie'] = merged_df['Weather'].apply(categoriser_meteo)

# Afficher les premières lignes pour vérifier
merged_df

Je regarde si la catégorisation avec le dictionnaire fonctionne correctement

In [ ]:
# Compteur pour chaque catégorie
compteur_categories = {categorie: 0 for categorie in categories}

# Fonction pour catégoriser et compter
def compter_categories(L):
    for texte in L:
        texte_lower = texte.lower()
        for categorie, mots_cles in categories.items():
            if any(mot in texte_lower for mot in mots_cles):
                compteur_categories[categorie] += 1
                break  # Arrête de chercher une fois qu'une catégorie est trouvée

# Appliquer la fonction de comptage
compter_categories(L)

# Afficher les résultats
for categorie, compteur in compteur_categories.items():
    print(f"{categorie}: {compteur}")

In [ ]:
pd.reset_option('display.max_rows')
merged_df

J'enlève la colonne 'Weather' maintenant que j'ai catégoriser la météo 

In [ ]:
vainqueur = pd.read_csv('winners.csv')
vainqueur

Je renomme les colonnes correctement pour procéder à la fusion des 2 bases de données

In [ ]:
vainqueur.rename(columns={vainqueur.columns[0]: 'annee'}, inplace=True)
vainqueur.rename(columns={vainqueur.columns[1]: 'numero_etape'}, inplace=True)
vainqueur

In [ ]:
# Fusionner les deux dataframes en utilisant les colonnes 'annee' et 'numero_etape' comme clés
table_etape = pd.merge(vainqueur, merged_df, on=['annee', 'numero_etape'], how='inner')
# Enregistrez le résultat dans un nouveau fichier CSV
table_etape.to_csv('table_etape', index=False)  
table_etape